# Question 1

## (i)
The primal problem is:
$$\underset{\pi\geq0}{\max}\sum_{x,y}\pi_{xy}\Phi_{xy}$$
s.t.
$$\sum_{x}\pi_{xy} \leq 1 \quad \forall y \in \{1, 2, 3, 4\}$$
$$\sum_{y}\pi_{xy} \leq 1 \quad \forall x \in \{1, 2, 3\}$$

In [1]:
using Gurobi, JuMP

In [37]:
Φ = [1 3 2 3; 2 5 4 1; 2 1 2 0]

m = Model(solver=GurobiSolver(Presolve=0, OutputFlag=0))

@variable(m, π[1:3, 1:4] >= 0)
@constraint(m, marginal_y[y=1:4], sum(π[x, y] for x in 1:3) <= 1)
@constraint(m, marginal_x[x=1:3], sum(π[x, y] for y in 1:4) <= 1)
@objective(m, Max, sum(sum(π[x, y] * Φ[x, y] for x in 1:3) for y in 1:4))

solve(m)

:Optimal

Academic license - for non-commercial use only


## (ii)

In [28]:
getvalue(π)

3×4 Array{Float64,2}:
 0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0

## (iii)

In [35]:
getdual(marginal_x)

3-element Array{Float64,1}:
 3.0
 5.0
 2.0

In [34]:
getdual(marginal_y)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Intuitively, man 1 could threaten woman 4 to leave her for woman 2 (whom he needs to offer 0). Woman 2 cannot threaten man 2, because if she went to man 1, she would need to offer him 3 (leaves 0 for her), and man 3 would not want to match with her, again, she has no bargaining power. Man 3 can freely choose between woman 1 and 3 and extract the whole surplus.

## (iv)
$W$ increases by $\epsilon$ and the surplus goes to man 2.

## (v)
Nothing changes.

# Question 2
## (i)
Since $\Phi$ is twice continuously differentialbe, supermodularity is equivalent to $\frac{\partial^2 \Phi(x,y)}{\partial x \partial y \geq 0}$. For the $\Phi$ at hand:
$$\frac{\partial^2 \Phi(x,y)}{\partial x \partial y} = \kappa y^{\kappa -1} \geq 0$$
We can expect positive assortative matching.

## (ii)

Since $F_X(x) = 1-\big(1-\frac{\beta x}{B}\big)^\frac{1}{\beta}$, and the fact that the rank of $x$ and the rank of the matched $y$ agrees (because of the rearrangement theorem):

$$T(x) = y = A \big(1-\frac{\beta x}{B}\big)^{-\frac{\alpha}{\beta}}$$

## (iii)
The constraints of the dual problem are equivalent to:
$$v(y) = \underset{x}{\max} \{\Phi(x,y) - w(x) \}$$
which implies the necessary first order condition:
$$\frac{\partial \Phi}{\partial x}(x,y) = 1y^\kappa = A \Big(1-\frac{\beta x}{B}\Big)^{-\frac{\kappa \alpha}{\beta}} = w'(x)$$
$$\frac{A\big(\beta x - B\big) \Big(1-\frac{\beta x}{B}\Big)^{-\frac{\kappa \alpha}{\beta}}}{\beta - \kappa \alpha}$$

# Question 3
<span style="color:red">I am not sure about this question. The discription is confusing and has bad grammar.</style>
## (i)

The primal problem is:
$$\underset{\pi\geq0}{\max}\sum_{x,y}\mu_{xy}\Phi_{xy}$$
s.t.
$$\sum_{x}\mu_{xy} \leq m_y \quad \forall y$$
$$\sum_{y}\mu_{xy} \leq n_x \quad \forall x$$
The dual is:
$$\underset{u, v \geq 0}{\min}\sum_{x}n_x u_x + \sum_{y}m_y v_y$$
s.t.
$$u_x + v_y \geq \Phi_{xy} \quad \forall x,y$$

I am very unclear about what is meant by $(u_x, v_y)$. Is it $(u_x, v_y) \in \mathbb{R}^2$ or $(u_x, v_y) \in \mathbb{R}^{n_x + m_y}$? I think the first one. I see them as the payoffs for a man and a woman who are married to each other. <span style="color:red">But then again, the text calls both $(u_x, v_y)$ and $(u, v)$ "vector of stable payoffs", so maybe I am wrong!</style>



# Question 7

In [23]:
using Distributions
nbDraws = 1000
ϵ_iy = rand(MvNormal(zeros(6), 0.5 * eye(6) + 0.5 * ones(6,6)), nbDraws)'
s_y = [0.216191, 0.263915, 0.141353, 0.176405, 0.113032, 0.089103]
s_y = s_y .+ (1-sum(s_y))/6 #it needs to sum to one!

m = Model(solver=GurobiSolver(Presolve=0, OutputFlag=0))

@variable(m, π[1:nbDraws, 1:6] >= 0)
@constraint(m, marginal_y[y=1:6], sum(π[x, y] for x in 1:nbDraws) == s_y[y])
@constraint(m, marginal_x[x=1:nbDraws], sum(π[x, y] for y in 1:6) == 1/nbDraws)
@objective(m, Max, sum(sum(π[x, y] * ϵ_iy[x, y] for x in 1:nbDraws) for y in 1:6))

solve(m)

Uhat_y = - getdual(marginal_y) + getdual(marginal_y)[end]

6-element Array{Float64,1}:
  0.34674  
  0.424859 
  0.127815 
  0.217393 
 -0.0423978
  0.0      

Academic license - for non-commercial use only


# Question 8
## (a)

The dual is:
$$\underset{\phi}{\max}\sum_{x \in X}n_x \phi_x - \sigma \sum_{(x,y) \in A} \exp \Big(\frac{\nabla_{xy} \phi - c_{xy} - \sigma}{\sigma}\Big)$$